In [1]:
import hashlib
import os
import random
import unittest

from key import fe, ECPubKey, P_384, P_384_G, P_384_ORDER

CUBERT2 = fe(2).cubert()

def Solve_Quartic_Equation_Over_Fq(a, b, c, d, e):
    # require p mod 3 = 2 and p mod 4 = 3
    # a,b,c,d,e all field element
    result = []
    delta1 = c**2 - fe(3) * b * d + fe(12) * a * e
    delta2 = fe(2) * c**3 - fe(9) * b * c * d + fe(27) * a * d**2 + fe(27) * b**2 * e - fe(72) * a * c * e
    tmp1 = -fe(4) * delta1**3 + delta2**2
    if not tmp1.is_square():
        return []
    tmp2 = delta2 + tmp1.sqrt()
    tmp3 = tmp2.cubert()
    delta = CUBERT2 * delta1 / (fe(3) * a * tmp3) + tmp3 / (fe(3) * CUBERT2 * a)
    
    L = -b / (fe(4) * a)
    
    tmp4 = b ** 2 / (fe(4) * a**2) - fe(2) * c / (fe(3) * a) + delta
    if not tmp4.is_square() or tmp4 == fe(0):
        return []
    tmp5 = tmp4.sqrt()
    
    M = tmp5 / fe(2)
    
    R1 = b ** 2 / (fe(2) * a**2) - fe(4) * c / (fe(3) * a) - delta
    R2 = (-b ** 3 / (a**3) + fe(4) * b * c / (a**2) - fe(8) * d /a) / (fe(4) * tmp5)
    Tstx12 = R1 - R2
    Tstx34 = R1 + R2
    if Tstx12.is_square():
        R = (Tstx12.sqrt()) / fe(2)
        x1 = L - M - R
        x2 = L - M + R
        result.append(x1)
        result.append(x2)
    if Tstx34.is_square():
        R = (Tstx34.sqrt()) / fe(2)
        x3 = L + M - R
        x4 = L + M + R    
        result.append(x3)
        result.append(x4)
    
    return result

def forward_map(u):
    """Forward mapping function

    Parameters:
        u (of type fe) : any field element
    Returns:
        fe, fe : affine X and Y coordinates of a point on the p-256 curve
    """
    a = fe(P_384.a)
    b = fe(P_384.b)
    v = (fe(3)*a - u**4) / (fe(6) * u)
    t = (v**2 - b - u**6 / fe(27))
    x = t.cubert() + u**2 / fe(3)
    y = u * x + v
    return x, y

def reverse_map(x, y, i):
    """Reverse mapping function

    Parameters:
        fe, fe : X and Y coordinates of a point on the secp256k1 curve
        i      : integer in range [0,3]
    Returns:
        u (of type fe) : such that forward_map(u) = (x,y), or None.

        - There can be up to 4 such inverses, and i selects which formula to use.
        - Each i can independently from other i values return a value or None.
        - All non-None values returned across all 4 i values are guaranteed to be distinct.
        - Together they will cover all inverses of (x,y) under forward_map.
    """
    a = fe(1)
    b = fe(0)
    c = -fe(6) * x
    d = fe(6) * y
    e = -fe(3) * fe(P_384.a)
    lst = Solve_Quartic_Equation_Over_Fq(a, b, c, d, e)
    if len(lst) > i:
        return lst[i]
    return None

    
def encode(P, randombytes):
    # P -> u, v; forward_map(u)+forward_map(v) = P; 
    count = 0
    while True:
        # Random field element u and random number j is extracted from
        # SHA256("secp256k1_ellsq_encode\x00" + uint32{count} + rnd32 + X + byte{Y & 1})
        m = hashlib.sha256()
        m.update(b"SWU_P-256_encode\x00")
        m.update(count.to_bytes(4, 'little'))
        m.update(randombytes)
        m.update(P[0].to_bytes(48, byteorder='big'))
        m.update((P[1] & 1).to_bytes(1, 'big'))
        hash = m.digest()
        u = fe(int.from_bytes(hash, 'big'))
        count += 1
        if count == 1:
            branch_hash = hash
            continue

        ge = forward_map(u)
        # convert ge to jacobian form for EC operations
        ge = (ge[0].val, ge[1].val, 1)
        T = P_384.negate(ge)
        Q = P_384.add(P_384.affine(P), P_384.affine(T))
        Q = P_384.affine(Q)
        if P_384.is_infinity(Q):
            Q = T
        j = (branch_hash[(count-2) >> 2] >> (((count-2) & 3) << 1)) & 3  # 0~3 randomness
        x, y, z = Q
        v = reverse_map(fe(x), fe(y), j)
        if v is not None:
            x1, y1 = forward_map(u)
            x2, y2 = forward_map(v)
            Sum = P_384.add((x1.val, y1.val, 1),(x2.val, y2.val, 1))
            Sum = P_384.affine(Sum)
            if (P[0] == Sum[0] and P[1] == Sum[1]):
                return u, v

def decode(u, v):
    # u, v -> P
    ge1 = forward_map(u)
    ge2 = forward_map(v)
    # convert ge1 and ge2 to jacobian form for EC operations
    T = ge1[0].val, ge1[1].val, 1
    S = ge2[0].val, ge2[1].val, 1
    P = P_384.add(T, S)
    if P_384.is_infinity(P):
        P = T
    P = P_384.affine(P)
    return P

import secrets

def generate_random_bytes(num_bytes):
    return secrets.token_bytes(num_bytes)

def encode_bytes(x, P, LT):
    k = secrets.randbelow((P * (2 ** LT) - x) // P)
    return x + k * P

def decode_bytes(enc, P, LT):
    return enc % P

LP = P_384.p.bit_length()
LT = 48
print(LP)

384


In [5]:
# for i in range(1000):
#     m = secrets.randbelow(P_384_ORDER)
#     A = P_384.affine(P_384.mul([(P_384_G, m)]))
#     ge = (A[0], A[1], A[2])
#     ge = P_384.affine(ge)

#     random_bytes = generate_random_bytes(32)
#     u, v = encode(ge, random_bytes)
#     P1 = decode(u, v)

#     assert ge[0]==P1[0] and ge[1] == P1[1]

In [6]:
# for i in range(1000):
#     m = secrets.randbelow(P_384_ORDER)
#     A = P_384.affine(P_384.mul([(P_384_G, m)]))
#     ge = (A[0], A[1], A[2])
#     random_bytes = generate_random_bytes(32)
    
#     u, v = encode(ge, random_bytes)
#     u_ = encode_bytes(u.val, P_384.p, LT)
#     v_ = encode_bytes(v.val, P_384.p, LT)
#     u = decode_bytes(u_, P_384.p, LT)
#     v = decode_bytes(v_, P_384.p, LT)
#     x, y, _ = decode(fe(u), fe(v))
#     assert ge[0] == x and ge[1] == y

In [4]:
All = [] 
for i in range(133334):
    m = secrets.randbelow(P_384_ORDER)
    A = P_384.affine(P_384.mul([(P_384_G, m)]))
    ge = (A[0], A[1], A[2])
    random_bytes = generate_random_bytes(48)
    u, v = encode(ge, random_bytes)
    u_ = encode_bytes(u.val, P_384.p, LT)
    v_ = encode_bytes(v.val, P_384.p, LT)
    All.append(u_.to_bytes(54, 'big'))
    All.append(v_.to_bytes(54, 'big'))
with open('output_file', 'wb') as file:
    for bit_string in All:
        file.write(bit_string)